<a href="https://colab.research.google.com/github/malinphy/q_17/blob/main/nih_patent_biogpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install transformers -q
!pip install sacremoses -q
# !pip install scann -q

In [3]:
import numpy as np 
import os 
import pickle
from transformers import AutoTokenizer, BioGptModel
import torch

In [4]:
os.listdir('drive/MyDrive/transfer/patent/patent_raw/nih')

['nih_patents_ada_embeddings.json',
 'embeddings.npy',
 'nih_title',
 'nih_abstract',
 'nih_data.ipynb']

In [5]:
titles_file = 'drive/MyDrive/transfer/patent/patent_raw/nih/nih_title'
file = open(titles_file, 'rb')
title_raw = pickle.load(file)
file.close()

abstract_file = 'drive/MyDrive/transfer/patent/patent_raw/nih/nih_abstract'
file = open(abstract_file, 'rb')
abstract_raw = pickle.load(file)
file.close()

In [6]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/biogpt");
model = BioGptModel.from_pretrained("microsoft/biogpt");

Some weights of the model checkpoint at microsoft/biogpt were not used when initializing BioGptModel: ['output_projection.weight']
- This IS expected if you are initializing BioGptModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BioGptModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [7]:
def normalizer(x):     
    normalized_vector = x / np.linalg.norm(x)
    return normalized_vector

class BioGptModel:
    
    def __init__(self, model, tokenizer):
        # self.sentence = sentence
        self.model = model
        self.tokenizer = tokenizer

    def sentence_vectors(self,sentence):
        inputs = self.tokenizer(sentence, return_tensors = 'pt')
        w_vectors = self.model(**inputs)

        # return w_vectors
        token_embeddings = w_vectors[0] #First element of model_output contains all token embeddings
        input_mask_expanded = inputs.attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
        return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

gpt_class = BioGptModel(model, tokenizer)
     


def sentence_encoder(data):
    vectors = []
    normalized_vectors = []
    for i in range(len(data)):
        sentence_vectors = gpt_class.sentence_vectors(data[i]).detach().numpy()
        vectors.append(sentence_vectors)
        normalized_vectors.append(normalizer(sentence_vectors))

    vectors = np.squeeze(np.array(vectors))
    normalized_vectors = np.squeeze(np.array(normalized_vectors))

    return vectors, normalized_vectors    

In [ ]:
abst_vectors, abst_vectors_norm = sentence_encoder(abstract_raw) 
title_vectors, title_vectors_norm = sentence_encoder(title_raw) 

In [ ]:
np.save('drive/MyDrive/transfer/patent/patent_raw/nih/data_nih_abstarct_biogpt.npy',abst_vectors)
np.save('drive/MyDrive/transfer/patent/patent_raw/nih/data_nih_abstarct_biogpt_norm.npy',abst_vectors_norm)

np.save('drive/MyDrive/transfer/patent/patent_raw/nih/data_nih_title_biogpt.npy',title_vectors)
np.save('drive/MyDrive/transfer/patent/patent_raw/nih/data_nih_title_biogpt_norm.npy',title_vectors_norm)